# Klassifisering av boligverdier med "California housing" datasett

## Litt om data: se eks. 7.2.7 "California housing data" på 
https://scikit-learn.org/stable/datasets/real_world.html

Bakgrunnsinfo om data fra den siden: \
<br>
    
$Data Set Characteristics:$

Number of Instances:
20640

Number of Attributes:
8 numeric, predictive attributes \
1 categorical predictive attribute \
and a categorical target variable (generated below)

Attribute Information: 

**longitude**  &emsp;    block group longitude \
**latitude**   &emsp; &emsp;   block group latitude  \
**housing_median_age**   &emsp;     median house age in block group \
**average_rooms**    &emsp;   average number of rooms per household \
**average_bedrooms**  &emsp;   average number of bedrooms per household \
**population**  &emsp;   block group population \
**average_occupancy**    &emsp;   average number of household members \
**median_income**    &emsp;&emsp;     median income in block group \
**ocean_proximity** &emsp;&emsp;     promixity to ocean ["'NEAR BAY'", "'1H OCEAN'", 'INLAND', "'NEAR OCEAN'", 'ISLAND']


Target variable: \
**house_value_categ** &emsp;&emsp;     categorical variable for the median value of a house in dollars - 


This dataset was obtained from the StatLib repository. https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is a categorical value for the median house value for California districts in dollars. It takes values .

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

A household is a group of people residing within a home. Since the average number of rooms and bedrooms in this dataset are provided per household, these columns may take surprisingly large values for block groups with few households and many empty houses, such as vacation resorts.

**NB:** SSBs Dapla-system tillater ikke direkte nedlasting av datasetter fra eksterne kilder som ikke er inkludert som en del av sklearn-installasjonen. Følgende eksempelkode fra scikit-learn.org vil ikke fungere:

- from sklearn.datasets import fetch_openml 
- california_housing = fetch_openml(data_id='43939', parser="auto")

Dette datasettet var lastet ned fra openML.org utenfor Dapla, fikk noe parsing og redigering, og ble kopiert til Dapla.

## Referanse til Python sklearn pakka:

https://scikit-learn.org/stable/api/index.html 

også

https://scikit-learn.org/stable/supervised_learning.html

# Initialiseringer

In [ ]:
import pandas as pd
import numpy as np
import dapla as dp
import time
import math

import statistics
import sys

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, scale

import pickle

In [ ]:
verbose=True

In [ ]:
data_sti='data/'
modell_sti='modeller/'
resultat_sti='resultater/'

In [ ]:
pd.set_option('display.precision', 2)

# Lese inn og forberede data

In [ ]:
housing=pd.read_csv('CA_housing_1990.csv', sep=';')

# samme bearbeiding som i regresjonseksempelet: 
# 1) fjerne kategorisk forklaringsvariabel ocean_proximity
# 2) fjerne punkter med manglende verdier
housing=housing.drop('ocean_proximity', axis=1)
housing = housing[~housing['average_bedrooms'].isna()]


In [ ]:
housing

In [ ]:
housing.hist(column='median_house_value', bins=50)

In [ ]:
# vi lager en egendefinert kategorisk variabel som beskriver median_house_value som:
#            < 130000           =>    'Low'
#    130000 <= ... <= 240000    =>    'Medium'
#    240000 <  ... < 490000     =>    'High' 
#            >= 490000          =>    'Very pricy'

thresh_lowmed=130000
thresh_medhigh=240000
thresh_veryhigh=490000

def dollars_to_categ(house_vals):
    
    house_categs= np.select(
    [   house_vals<thresh_lowmed,
        house_vals.between(thresh_lowmed, thresh_medhigh, inclusive='both'), 
        house_vals.between(thresh_medhigh, thresh_veryhigh, inclusive='neither'),
        house_vals>=thresh_veryhigh   ], 
    [   'Low', 
        'Medium',
        'High',
        'Very pricy'    ])
    return(house_categs)
    
housing['house_value_categ']= dollars_to_categ(housing['median_house_value'])   
housing    

In [ ]:
housing=housing.drop('median_house_value', axis=1)

# Lage X, y 

In [ ]:
X=housing.drop('house_value_categ', axis=1)

In [ ]:
# legge X_ til kolonnenavn for X-variabler

X.columns='X_'+X.columns
X.columns

In [ ]:
X

In [ ]:
y= housing.loc[:,'house_value_categ'].values
y

In [ ]:
# litt utforskning av data

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# summary statistics
X.describe()

In [ ]:
pd.DataFrame(y).value_counts()

# Dele i trenings- og testsetter

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

# Trene modell 

In [ ]:
# vi prøver først en support vector machine

model = SVC()

model.fit(X_train, y_train)


### Litt inspisering

hvordan klassifiserer modellen et punkt med gjennomsnittlige verdier for alle X variabler?

In [ ]:
means = X.describe().loc[['mean']]
means

In [ ]:
model.predict(means)

# Test modellen

In [ ]:
# lage prediksjoner på testdata

y_pred = model.predict(X_test)
y_pred

In [ ]:
# lage en kopi for senere

y_pred_SVM = y_pred

### Vurdere modellen

In [ ]:
# beregne nøyaktigheten (prosent av prediksjoner som er riktige) 
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
# beregne F1-score

# F1: geometric mean of recall and precision, 
#     weighted averaged over the categories for the outcome variable y

f1 = f1_score(y_test, y_pred, average="weighted")
print(f"F1 score: {f1:.2f}")

In [ ]:
# Vis 'classification report', som gir mer detaljerte mål
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

In [ ]:
# hvis ønsket, skriv ut klassifiserings rapport til en tekst fil:
print(report, file=open('classification_report_CA_housing_values_SVM.txt', 'w'))

### Vurdere ytelsen i strata

In [ ]:
# sjekke nøyaktigheten i strata hvor hus er nyere eller eldre
nyere = X_test['X_housing_median_age']<=20

accuracy_new = accuracy_score(y_test[nyere], y_pred[nyere])
print(f"Accuracy, housing_median_age<=20: {accuracy_new:.2f}")


# Prøv andre algoritmer

In [ ]:
# nå prøver vi en random forest

model=RandomForestClassifier()

# tilpasning, sjekk treningstid
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
print(f"Model trained in {end_time-start_time:.1f} seconds")

# sjekke nøyaktigheten på testsettet
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
# lage en kopi av modellen for lagring
model_RF=model 

### Prøv med dine egne modelldefinisjoner her!

# Støttefunksjoner for testing

In [ ]:
# lag en funksjon for å sjekke nøyaktighet av en modell

def sjekke_noyaktighet(mymodel):
    mymodel.fit(X_train, y_train)
    y_pred = mymodel.predict(X_test)
    accuracy=accuracy_score(y_test, y_pred)
    print("Testing model ", mymodel, "\naccuracy=", round(accuracy,2))
    return None


In [ ]:
# prøv logistisk regresjon
# øke maksimalt antall iterasjoner for å hjelpe med konvergens

mymodel=LogisticRegression(max_iter=1000)
sjekke_noyaktighet(mymodel)

ingen konvergens selv med den høyere maksimalt antallen iterasjoner\
så prøver vi å skalere X i dataene først

In [ ]:
mymodel = Pipeline([   ('scale', StandardScaler()),
                  ('LR', LogisticRegression(max_iter=1000)) ])
sjekke_noyaktighet(mymodel)

nå har vi konvergens og bedre ytelse!

hjelper skalering med de andre algoritmene vi har prøvd også?

In [ ]:
# support vector machine (SVM)

mymodel = Pipeline([   ('scale', StandardScaler()),
                  ('SVM', SVC()) ])
sjekke_noyaktighet(mymodel)

In [ ]:
# random forest

mymodel = Pipeline([   ('scale', StandardScaler()),
                  ('RF', RandomForestClassifier()) ])
sjekke_noyaktighet(mymodel)